In [1]:
# импорты
import nltk
import re
import pickle
from string import punctuation
import math
from tqdm import tqdm_notebook
import csv
import gensim

import pymystem3
m = pymystem3.Mystem() #для использования лемматизации

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

C:\Users\Aleksandra\AppData\Roaming\Python\Python36\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def unpack(data):
    input = open(data, 'rb')
    obj = pickle.load(input)
    input.close()
    return obj

# загружаем частоты лем униграмм
unigrams = unpack('1stemgrams.data')
# убрали пробел из начала слов
unigrams = {w[1:]:f for w,f in unigrams.items()}

In [3]:
bigrams = unpack('2grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
bigrams  = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in bigrams.items()}

In [4]:
list(bigrams.keys())[:10]

['дальнейший допрос',
 'оканчивать преступление',
 'развитие христианство',
 'и затонуть',
 'подразумевать она',
 'и однозначный',
 '2 введение',
 'желать услышать',
 'являться недооценка',
 'закрепляться по']

In [5]:
trigrams = unpack('3grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
trigrams = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in trigrams.items()}

In [6]:
list(trigrams.keys())[:10]

['в лифт я',
 'что сам русский',
 'почему ты думать',
 'фестиваль в кольмар',
 'твердость и прочность',
 'тот число руководитель',
 'судьба решать иначе',
 'мысль рождаться в',
 'мой учение то',
 'да только что']

In [7]:
# читает из файла, убирает двойные пробелы и ручные переносы, последний \n
def reading(file):
    f = open('texts\\{}.txt'.format(file), 'r', encoding='utf-8')
    text = f.read()
    text = text.replace('  ', ' ')
    text = text.replace('-\n', '')
    if text[-1] == '\n': # убираем последний \n, если такой есть 
        text = text[:-1]
    f.close()
    return text

In [8]:
def minimummaker(): #ф-ция превращения текстового файла с минимумом в питоновский список. Запаковка списка
    with open('min.txt', encoding='utf-8') as file:
        lemtokens = [morph.parse(i)[0].normal_form for i in re.findall('\w+', file.read().lower())] #делаем список лемм слов
        minimum = list(set(lemtokens)) #убираем повторы
        output = open('minimum.pkl', 'wb')
        pickle.dump(minimum, output, 2)
        output.close()

def loadminimum(): #распаковка cписка с минимумом. Возвращает неупорядоченный список
    input = open('minimum.pkl', 'rb')
    minimum = pickle.load(input)
    input.close()
    return minimum

#minimummaker()
minimum = loadminimum()
#print(minimum[-50:]) #проверка работы
#print(len(minimum)) #2549 слов в списке

In [9]:
# загрузка модели
def model_loading(file):
    model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)
    model.init_sims(replace=True)
    print('Done!') 
    return model

model = model_loading('news_upos_cbow_600_2_2018.vec') #на загрузку тратится минут 10

Done!


In [10]:
# загрузка словаря ASIS
with open('syns.data', 'rb') as f:
     asis = pickle.load(f)

In [11]:
# Таблица конверсии в UPoS из тэгов Mystem
# словарь, переводящий теги mystem в universal теги моделей
mystem_tags = {'A' : 'ADJ',
       'ADV' : 'ADV',
       'ADVPRO' : 'ADV',
       'ANUM' : 'ADJ',
       'APRO' : 'DET',
       'COM' : 'ADJ',
       'CONJ' : 'SCONJ',
       'INTJ' : 'INTJ',
       'NONLEX' : 'X',
       'NUM' : 'NUM',
       'PART' : 'PART',
       'PR' : 'ADP',
       'S' : 'NOUN',
       'SPRO' : 'PRON',
       'UNKN' : 'X',
       'V' : 'VERB',
       'X' : 'X',
      'PROPN' : 'PROPN'} #последних 2 тегов в майстеме нет, но они задаются в классе для слов в соответсвующими пометами

# словарь, переводящий теги пайморфи в universal теги моделей
pymorphy_tags = {'ADJF':'ADJ',
    'ADJS' : 'ADJ',
    'ADVB' : 'ADV',
    'COMP' : 'ADV',
    'GRND' : 'VERB',
    'INFN' : 'VERB',
    'NOUN' : 'NOUN',
    'PRED' : 'ADV',
    'PRTF' : 'ADJ',
    'PRTS' : 'VERB',
    'VERB' : 'VERB'}

In [12]:
class Token():
    def __init__(self, w):
        
        self.num = None # номер в тексте
        self.complexity = None # сложность слова
        
        # три варианта инициализации: 
        ## из анализа текста, 
        ## из уже имеющегося объекта (для дочернего класса ComplexWord) 
        ## из строки
        
        if isinstance(w, dict): # если получили результат работы mystem
            
            self.text = w['text']  # сам токен
            self.len = len(w['text']) # его длина
            
            # определяет, сделан ли анализ и, соответственно, рассматривать ли как слово, требующее упрощения
            gram = w.get('analysis')
            if gram:
                self.lexem = gram[0]['lex']  # лемма
                
                if not self.named_entity(gram[0]):  # именованная сущность или нет
                    self.pos = self.pos_tag(gram[0]['gr'])  # часть речи
                else:
                    self.pos = 'PROPN' # universal tag for named entity - у майстема таких нет
                
                    
            elif any(p in w['text'] for p in punctuation+'–«»'): # если это знак пунктуации (может быть с пробелом!)
                self.lexem = '_PUNKTUATION_'
                self.pos = None
            
            elif not re.findall('\S',w['text']): # если это только пробельные символы
                self.lexem = '_SPACE_'
                self.pos = None
                
            # остальное - неизвестная и ненужная ерунда?
            else:
                self.lexem = '_UNK_'
                self.pos = 'X' # universal tag for unknown
            
            
        elif isinstance(w, Token): # для определения объектов дочернего класса ComplexWord
            self.text = w.text
            self.num = w.num
            self.lexem = w.lexem 
            self.len = w.len
            self.pos = w.pos
            self.complexity = w.complexity
            
        
        elif isinstance(w, str): # если хотим как класс токен определить строку, полученную из словаря или модели
            self.text = w
            self.pos = None
            self.lexem = w
            self.len = len(w)
            self.num = None
            self.complexity = None
            
        
    # вытаскивает часть речи из разбора майстем
    def pos_tag(self,gram):
        if ',' in gram:
            gram = gram.split(',')[0]
        if '=' in gram:
            gram = gram.split('=')[0]
        return gram
        
    # определяет по тегам, является ли именованной сущностью
    def named_entity(self,gram):
        markers = {'сокр': ' - сокращение', 'фам': ' - фамилия', 'имя': ' - имя собственное', 'гео': ' - название места', }
        if any(m in gram['gr'] for m in markers.keys()):
            return True
        else:
            return False

    def complexity_params(self, param = 'freq'):
        # если по частотности
        if param == 'freq':
            self.complexity = unigrams.get(self.lexem, 0)
            
        # если по коэффициенту информативности. Отрицательное значение. Чем он меньше, тем сложнее
        elif param == 'inf':
            self.complexity = math.log((unigrams.get(self.lexem, 0)+1)/(sum(f for f in unigrams.values())+1))
        
    
    def is_complex(self, threshold = '600', use_min = False, len_threshold = 1000):
        exceptions = ['_PUNKTUATION_', '_SPACE_', '_UNK_']
        # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
        if not any(exception in self.lexem for exception in exceptions) and self.pos not in ['PROPN']:
            
            # если показатель сложности - вхождение в минимум
            if use_min:
                if self.lexem not in minimum:
                    return True
                else:
                    return False

            # если показатель сложности - пороговое значение сложности
            # также может использоваться длина. По умолчанию слишком большое - 1000 (т.е. этот параметр не учитывается)
            else:
                if self.complexity < float(threshold) or self.len > len_threshold:
                    return True
                else:
                    return False
        else:
            return False
    
    def convert_universal(self):
        if self.pos in mystem_tags:
            self.pos = mystem_tags[self.pos]
        else:
            self.pos = 'X' # Х - universal тег для неизвестных слов
        return self
'''Пример вывода атрибутов объектов класса 
token.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):

223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False
224   _SPACE_ None -19.06687873346149 False
225 является являться V -7.502177090710664 False
226   _SPACE_ None -19.06687873346149 False
227 должником должник S -11.184186527172464 True
228   _SPACE_ None -19.06687873346149 False
229 банка банк S -9.300701131889967 False
230 .  _PUNKTUATION_ None -19.06687873346149 False
'''

'Пример вывода атрибутов объектов класса \ntoken.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):\n\n223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False\n224   _SPACE_ None -19.06687873346149 False\n225 является являться V -7.502177090710664 False\n226   _SPACE_ None -19.06687873346149 False\n227 должником должник S -11.184186527172464 True\n228   _SPACE_ None -19.06687873346149 False\n229 банка банк S -9.300701131889967 False\n230 .  _PUNKTUATION_ None -19.06687873346149 False\n'

In [13]:
'''# вытаскивает часть речи из разбора майстем
def pos(gram):
    if ',' in gram:
        gram = gram.split(',')[0]
    if '=' in gram:
        gram = gram.split('=')[0]
    return gram
'''

#анализ текста 
def text_structuring(text, param, threshold, use_min):
    # анализирует текст 
    analysis = m.analyze(text)
    tokens = []
    for i, w in enumerate(analysis): # состаляем список объектов Tokens
        token = Token(w)
        token.num = i # добавляем токену в атрибуты его номер в тексте
        token.complexity_params(param) # переопределяем сложность на основе выбранного параметра
        token.convert_universal() # превращаем POS в universal формат
        print(token.num, token.text, token.lexem, token.pos, token.complexity, token.is_complex(threshold, use_min))
        tokens.append(token)
    return tokens

In [14]:
# ПАРАМЕТРЫ анализа слов
complexity_type = 'inf'
global_threshold = -8.5
use_min=True

In [15]:
text = reading('news3')
tokens = text_structuring(text, complexity_type, global_threshold, use_min)

0 В в ADP -3.486155131943022 False
1   _SPACE_ X -19.06687873346149 False
2 Астраханской астраханский ADJ -11.87544940342511 True
3   _SPACE_ X -19.06687873346149 False
4 области область NOUN -8.188095962474803 False
5   _SPACE_ X -19.06687873346149 False
6 ветерану ветеран NOUN -10.97250028848853 True
7   _SPACE_ X -19.06687873346149 False
8 ВОВ вов PROPN -14.907995650101817 False
9   _SPACE_ X -19.06687873346149 False
10 вернули вернуть VERB -9.947667294896409 False
11   _SPACE_ X -19.06687873346149 False
12 похищенные похищать VERB -11.006654493020532 True
13   _SPACE_ X -19.06687873346149 False
14 медали медаль NOUN -10.586972126831268 True
15 
 _SPACE_ X -19.06687873346149 False
16 
 _SPACE_ X -19.06687873346149 False
17 В в ADP -3.486155131943022 False
18   _SPACE_ X -19.06687873346149 False
19 Астраханской астраханский ADJ -11.87544940342511 True
20   _SPACE_ X -19.06687873346149 False
21 области область NOUN -8.188095962474803 False
22   _SPACE_ X -19.06687873346149 False
23 ве

In [16]:
class Substitution(Token):
    def __init__(self, w):
        super().__init__(w)
        self.similarity = None
        self.fitness = None
    
    # для слов из словаря и тезауруса: определяем тег пайморфи, переводим в формат universal - так быстрее, чем майстемом
    def tagging(self, w):
        tag = morph.parse(w)[0].tag.POS
        if tag in pymorphy_tags:
            return pymorphy_tags[tag]
        else:
            return 'X' # Х - universal тег для неизвестных слов

    # в случае, когда Substitution получено не через модель, и его близость неизвестна
    def measuse_similarity(self, target):
        #print(self.lexem, target.lexem, self.pos, target.pos)
        subst_query = str(self.lexem+'_'+self.pos)
        target_query = str(target.lexem+'_'+target.pos)
        if subst_query in model and target_query in model:
            self.similarity = model.similarity(subst_query, target_query)
        # может понадобиться return self
    
    # приписываем атрибуты словам, взятым из словаря или тезауруса
    def setting_atr(self, target):
        self.pos = self.tagging(self.lexem)
        self.complexity_params(complexity_type)
        self.measuse_similarity(target)
        
    def language_model(self):
        pass

In [17]:
class Complex_word(Token):
    def __init__(self, w):
        super().__init__(w)
        self.substituts = None
        self.place = None
        self.context = None
        self.easier = []
        
    # список замен, в зависимости от выбранной базы
    def search_substituts(self, base_type='model'):
        
        # поиск по модели
        def model_search(lexem, pos):
            query = str(lexem+'_'+pos)
            #print(query)
            if query in model:
                # формируем список квазисинонимов той же части речи
                # при этом превращаем их в объекты соответствующего класса
                syn_tokens = []
                for syn, sim in model.most_similar(positive=[query]):
                    syn_text = syn[:syn.find('_')] # текст до части речи
                    
                    syn_tok = Substitution(syn_text) # из текстовой строки инициализируем объект класса
                    syn_tok.pos = syn[syn_tok.len+1:] # часть речи
                    
                    syn_tok.complexity_params(complexity_type) # сложность по функции в зависимости от выбранного параметра
                    
                    syn_tok.similarity = sim # а близость по параметру модели
                    
                    syn_tokens.append(syn_tok)
                    
                return syn_tokens
            else:
                return [] 

        # поиск по YARN
        def yarn_search(target, filepath = 'yarn-synsets1.csv'):
            with open(filepath, "r", newline="") as file: # постепенный просмотр файла с синсетами (множествами синонимов)
                reader = csv.DictReader(file, delimiter=';')
                lst = []
                for i,row in enumerate(reader):
                    cur_line = row['words'].split(';') # считываем колонку с синсетами
                    if len(cur_line)>1:
                        if target.lexem in cur_line:
                            del(reader)
                            for c in cur_line:
                                if ' ' not in c and c!=target.lexem: # формируем список однословных синонимов
                                    sub_tok = Substitution(c)
                                    
                                    sub_tok.setting_atr(target)
                                    
                                    if sub_tok not in lst:
                                        lst.append(sub_tok) 
                            #TODO: выделить неоднословные в отдельный класс и поискать их частотность по n-граммам?
                            break
                #print(lst)
                return lst 
        
        #поиск по ASIS
        def asis_search(target):
            if target.lexem in asis:
                lst = []
                for s in asis[target.lexem]:
                    if ' ' not in s: # формируем список однословных синонимов
                        sub_tok = Substitution(s)
                        sub_tok.setting_atr(target)
                        lst.append(sub_tok)
                return lst
            else:
                return []

        
        if base_type == 'model':
            self.substituts = model_search(self.lexem, self.pos)
            
        if base_type == 'yarn':
            self.substituts = yarn_search(self)
        
        if base_type == 'asis':
            self.substituts = asis_search(self)
            
        return self
    
    def find_easier(self, use_min = False, threshold = global_threshold):
        for sub in self.substituts:
            if not sub.is_complex(threshold = threshold, use_min = use_min):
                if sub.complexity > self.complexity:
                    self.easier.append(sub)
        return self
    
    def make_window(self, tokens, window = 10):
        context = [self]
        left_ind = self.num-1
        right_ind = self.num+1
        ind = 0
        # добавляем по одному слову слева и/или справа, пока не наберется window + само слово
        while len(context)<window+1:
            while left_ind >= 0:
                left_w = tokens[left_ind]
                left_ind-=1
                # проверка, что это слово
                if any(exception in left_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context[:0] = [left_w] #вставляем слово слева от цепочки
                    ind+=1 # индекс слова сдвигается
                    break

            while right_ind < len(tokens):
                right_w = tokens[right_ind]
                right_ind+=1
                # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
                if any(exception in right_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context.append(right_w) # справа от цепочки
                    break
        self.place = ind
        self.context = context
        return self

In [22]:
# Отбор сложных слов, превращение их в подкласс сложных слов и поиск замен
# несложные слова также остаются на своих местах
def selecting_complex(tokens, base_type=['yarn','model','asis'][0], threshold = global_threshold, use_min = False):
    complex_words = []
    for token in tokens:
        if token.is_complex(threshold, use_min):
            comp_token = Complex_word(token) # токен становится Сложным словом
            
            comp_token.search_substituts(base_type=base_type)
            
            complex_words.append(comp_token)
            
            # код для принтов
            
            '''
            if comp_token.substituts:
                for syn in comp_token.substituts:
                    print (comp_token.lexem, comp_token.complexity, base_type, ':', syn.lexem, syn.complexity, syn.similarity)
            '''
            comp_token.find_easier(threshold, use_min)
            if comp_token.easier:
                for syn in comp_token.easier:
                    print (comp_token.lexem, comp_token.pos, base_type, ':', syn.lexem, syn.pos, syn.complexity, syn.similarity)
            
        else:
            complex_words.append(token)
    return complex_words

complex_words = selecting_complex(tokens, 'yarn', threshold = global_threshold, use_min = False)

ветеран NOUN yarn : старик NOUN -8.345441216450578 0.23961324224267147
ветеран NOUN yarn : старик NOUN -8.345441216450578 0.23961324224267147
отечественный ADJ yarn : родной ADJ -9.113982160667819 0.020030045830958295
портал NOUN yarn : вход NOUN -9.731845917527988 0.0002885425674101231
портал NOUN yarn : дверь NOUN -7.711707098541537 0.04702153840192069
подозревать VERB yarn : думать VERB -7.108101460589846 0.03465083751933694
ветеран NOUN yarn : старик NOUN -8.345441216450578 0.23961324224267147
ветеран NOUN yarn : старик NOUN -8.345441216450578 0.23961324224267147
возникать VERB yarn : подниматься VERB -8.366785291997186 0.20159820914507315
возникать VERB yarn : появляться VERB -8.062597938932536 0.35273497014407196
возникать VERB yarn : начинаться VERB -8.153027397811316 0.3254855920577131
возникать VERB yarn : происходить VERB -7.775560500548489 0.4064410044068008
забирать VERB yarn : собирать VERB -8.770268596015583 0.24647246517422808
впоследствии ADV yarn : затем ADV -8.1780409

### Решить проблему нахождения более простых синонимов. Пока возвращает только пустоту при use_min = True (по видимому, слов нет в минимуме?). Сейчас структура не очень логичная: сложные слова в тексте - те, что не вошли в минимум, но отнесение к сложным/простым синонимов происходит на основании порогового значения

### теперь, когда для каждого сложного слова есть список его синонимов с нужными параметрами, можно делать модель на н-граммах
### контекстное окно: 2n слов (по n слева и справа), можно задавать. Вероятность на основе логарифма + Лапласса

In [28]:
from nltk.util import ngrams

In [35]:
ngrams_dict = {0: {}, 1: unigrams, 2: bigrams, 3: trigrams}

In [2]:
def prob(ngram, n):
    c1 = ' '.join(ngram[-n:]) # в числителе частота строки длины n
    c2 = ' '.join(ngram[-n:-1]) # в знаменателе - строки без последнего символа
    d = ngrams_dict.get(n) # для поиска числителя берем словарь n-грамм
    d2 = ngrams_dict.get(n-1) # для поиска знаменателя - словарь n-1-грамм
    V = len(unigrams)
    #len(ngrams_dict.get(n-1,len(unigrams))) # сглаживание лапласса: добавляем размер словаря знаменателя
    p1 = d.get(c1,1)
    p2 = d2.get(c2,1)
    print('\t',c1, '/', c2, ':' , p1, '/', p2, '(', p1+1, '/', p2+V, ')')
    return math.log(p1+1)-math.log(p2+V)

In [3]:
for token in complex_words:
    if isinstance(token, Complex_word):
        if token.easier: 
            # окна контекста строим для тех сложных слов, для которых есть варианты замен, которые проще, чем слово
            token.make_window(complex_words, window = 5)
            
            # вывод
            context_lem = [c.lexem for c in token.context]
            #print(token.text, token.place, ':', context_lem)
            
            # вычисляем вероятность цепочки
            
            # генерация 3-грамм и 2-грамм
            text_3grams = [n for n in ngrams(context_lem, 3)]
            
            # добавляем слева искусственные н-граммы для вычисления вероятности
            #text_3grams = [('', '', text_3grams[0][0]), ('', text_3grams[0][0], text_3grams[0][1])] + text_3grams
            
            #print(text_3grams)
            p_context = 1.0
            for ngram in text_3grams:
                
                p = math.log(prob(ngram, 3))
                #print(ngram, p)
                p_context+=p
                
            print('\nВероятность слова _{0}_ в контексте {1} = {2}\n'.format(token.text, context_lem, p_context))
            
            best_fitness = -1000
            best_sub = None
            for sub in token.easier:
                sub_context_lem = context_lem[:token.place]+[sub.lexem]+context_lem[token.place+1:]
                #сделать из этого куска функцию
                sub_3grams = [n for n in ngrams(sub_context_lem, 3)]
                # добавляем слева искусственные н-граммы для вычисления вероятности
                #sub_3grams = [('', '', sub_3grams[0][0]), ('', sub_3grams[0][0], sub_3grams[0][1])] + sub_3grams
            
                p_changed_context = 1.0
                for ngram in sub_3grams:

                    p = prob(ngram, 3)
                    #print(ngram, p)
                    p_changed_context+=p
                    
                fit='НЕТ'
                if p_changed_context>p_context:
                    fit='ДА'
                    sub.fitness = p_changed_context
                    if sub.fitness > best_fitness:
                        best_fitness = sub.fitness
                        best_sub = sub
                print('\nВероятность замены _{0}_ в контексте {1} = {2}, разница = {3}, меняем? - {4}\n'.format(sub.text, sub_context_lem, p_changed_context, p_context-p_changed_context, fit))
            
            if best_sub:
                print('Лучшее значение вероятности: {0}\n'.format(best_sub.lexem))
            

NameError: name 'complex_words' is not defined

In [44]:
bigrams.get('в астраханский',0)

186

In [1]:
import math
math.log(0.0)

ValueError: math domain error